In [1]:
import pickle
import powerlaw
import networkx as nx
import numpy as np
import os

In [2]:
def cos_angle(v1,v2):
    return np.dot(v1,v2)/np.linalg.norm(v1)/np.linalg.norm(v2)

In [3]:
def get_similarity_matrix(vocab, method, model, normed=False):
    if method == 'cos':
        f_method = cos_angle
    else:
        f_method = np.dot

    mat = np.zeros((len(vocab), len(vocab)))
    for i, word1 in enumerate(vocab):
        for j in np.arange(i+1, len(vocab)):
            word2 = vocab[j]
            mat[i,j] = f_method(
                model.word_vec(word1, use_norm=normed),
                model.word_vec(word2, use_norm=normed))
    return mat

### K-nn 

Constructing a directed graph from the w2v vectors. This method is introduced in Steyvers & Tenenbaum (2005) and is known as k-nn, since every word will have out-going connections to k other words that are determined as words whose vectors give the highest dot product.

In [4]:
def get_knn_edges(mat, vocab, fan_k):
    edges = []
    matT = mat + mat.T  # get a symmetric matrix
    for i, row in enumerate(matT):
        word = vocab[i]
        k = fan_k[word]
        k_assoc = [vocab[assoc] for assoc in np.argsort(row)[::-1][:k]]
        edges.extend(list(zip([word]*len(k_assoc), k_assoc)))
    return edges

### cs-method 
The method introduced in the Utsumi (2015), the neighbors of a word are computed based on the cumulative similarity 
ratio:

$\frac{\Sigma_{w_j \in V_i^N} \cos (w_i, w_j)}{\Sigma_{w_j \in V} \cos (w_i, w_j)} > R$

It's not entirely clear how they deterimened R_max, but I assume the formula was evaluated for every k, starting with 1 and when it crossed the threshold, that k was was selected. The threshold was probably hand-picked so that it produces <k> as in the association norms.|

max_k is set to 324 as that was the maximum number of connections for the directed USF networks, but in practice all k's are lower so that number is never reached:

In [5]:
def get_cs_edges(mat, vocab, R_max, max_k=324):
    directed_edges = []
    ks = []
    max_k = 324

    matT = mat + mat.T
    for i, row in enumerate(matT):
        word = vocab[i]

        total_sim = np.sum(row)
        if total_sim < 0.01:
            print(word, total_sim)

        for k in range(1, max_k):
            k_idx = np.argsort(row)[::-1][:k]
            kassoc_sim = np.sum(k_idx)
            R = kassoc_sim/total_sim
            if R > R_max:
                k_assoc = [vocab[assoc] for assoc in k_idx]
                directed_edges.extend(list(zip([word]*len(k_assoc), k_assoc)))
                break

        ks.append(k)
    print('Average degree:', np.mean(np.array(ks)))
    return directed_edges